## Project 3: Which age range in NYC makes the most 911 calls?
by Sophia Cain

# Introduction

**Datasets**

For this project, I will be using two different data sets from [NYC Open Data](https://opendata.cityofnewyork.us/) to explore the volume of 911 calls in New York in relation to age groups. The first data set is [NYPD Calls for Service](https://data.cityofnewyork.us/Public-Safety/NYPD-Calls-for-Service-Year-to-Date-/n2zq-pubd/about_data) which provides the date and borough of 911 calls made in New York. The second data set is [Projected Population 2010-2040 - Total By Age Groups](https://data.cityofnewyork.us/City-Government/Projected-Population-2010-2040-Total-By-Age-Groups/97pn-acdf/about_data) which gives population age data, broken down into 18 age cohorts, by NYC borough. 

**Analysis Question**

What NYC boroughs have the highest proportion of residents above 65 (the age to qualify for medicare) to the total population?

What NYC boroughs have the highest proportion of 911 calls to total population?

Is there overlap in the density of residents over 65 and 911 calls made?

Which NYC age range has the highest proportion of 911 calls to population in each borough?

**Columns that will (likely) be used**

For the Age/Population data frame:
- Borough
- Age Range
- Year/Date

For the 911 Call data frame:
- Borough
- Year/Date
- Number of call/incident date

**Hypothesis**

I hypothesize that boroughs with higher elderly populations (65+) will also have a higher volume of 911 calls annually.

# Step 1: Load in Data

I start by loading in the necessary packages to complete my analysis. Initially, this is pandas and the plotly code that ensures my visualization will sync with my github website.

I then load in the new data frames that I will be using to conduct this analysis. For the NYC population age data, I load it in as a CSV because the data frame is relativley small. For the NYC 911 data, this is a considerably larger data frame that includes geographical codes. Therefore, I chose to load this in through an API and then convern the JSON data into a data frame.

I use .head and .info to get more information on the type of data I am working with and how it will need to be cleaned.

In [12]:
import plotly.io as pio

pio.renderers.default = "notebook_connected+plotly_mimetype"

**NYC Borough Population by Age Range Data**

In [13]:
import pandas as pd

NYC_pop_age = pd.read_csv("NYC_pop_age.csv")

NYC_pop_age.head()

,Borough,Age,2010,2015,2020,2025,2030,2035,2040
0,NYC Total,0-4,"521,990","535,209","545,778","547,336","542,426","540,523","546,426"
1,NYC Total,15-19,"539,844","505,783","492,532","519,298","535,024","546,062","546,750"
2,NYC Total,20-24,"647,483","646,075","606,203","591,683","625,253","643,728","657,403"
3,NYC Total,25-29,"736,105","770,396","763,956","715,824","698,195","740,437","762,757"
4,NYC Total,30-34,"667,657","707,726","743,916","740,268","693,684","675,497","715,486"


In [14]:
NYC_pop_age.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114 entries, 0 to 113
Data columns (total 9 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Borough  114 non-null    object
 1   Age      114 non-null    object
 2   2010     114 non-null    object
 3   2015     114 non-null    object
 4   2020     114 non-null    object
 5   2025     114 non-null    object
 6   2030     114 non-null    object
 7   2035     114 non-null    object
 8   2040     114 non-null    object
dtypes: object(9)
memory usage: 8.1+ KB


In [15]:
NYC_pop_age["Age"].unique()

array(['0-4', '15-19', '20-24', '25-29', '30-34', '35-39', '40-44',
       '45-49', '50-54', '55-59', '60-64', '65-69', '70-74', '75-79',
       '80-84', '85+', 'Total', '5-9', '10-14'], dtype=object)

From this, I can see that all of the data is stored as objects. I also used the .unique function to see how many age ranges there are, so I can combine all of the 65+ age ranges.

In [16]:
NYC_pop_age["Borough"].unique()

array(['NYC Total', 'Bronx', 'Brooklyn', 'Manhattan', 'Queens',
       'Staten Island'], dtype=object)

I used the .unique function to see how the boroughs were named in this data.

**911 Calls by Borough in 2025**

**Disclaimer:** Initially I loaded in this data set through the API endpoint because it was too large to download as a CSV. The code for that can be found below:

import requests

response = requests.get("https://data.cityofnewyork.us/resource/n2zq-pubd.json")

NYC_911 = response.json()

df_911 = pd.DataFrame(NYC_911)

df_911

This code worked and loaded in the data, but it only loaded in the first 1000 rows of 911 calls. NYC has an average of 9 million 911 calls a year, therefore I came to the conclusion that this was not the entire data set. Upon furter inspection, I realized that there were actually 5.3 million rows of data in this data set. This makes sense because when I viewed the data dictionary, I was able to figure out that this data is updated quarterly. The 5.3 million rows of this dataset represents a little over the first 3 quarters of 2025 NYC 911 calls, from January to November. 

After attemping to read in the entire data set with 5.3 million rows, I realized that I would need to clean/filter the data while loading it in through the API. From the NYC Open Data website (and with the help of Sneha during office hours), I found more information on the [API documentation](https://dev.socrata.com/foundry/data.cityofnewyork.us/n2zq-pubd). This API lets you "combine filters together by stacking parameters on your URL or by using SoQL queries." I do not know SQL like that and therefore utilized AI to help me with the SQL part of the code needed to load in this data.

In [ ]:
import pandas as pd
import requests
from urllib.parse import quote_plus

base_url = "https://data.cityofnewyork.us/resource/n2zq-pubd.json"

# This code filters for the year 2025/
where_clause = (
    "incident_date >= '2025-01-01T00:00:00' AND incident_date < '2026-01-01T00:00:00'"
)

# Build SoQL query to group by boro_nm and count incidents (this was helped by AI)
soql_query = (
    "$select=boro_nm, count(*)&$where=" + quote_plus(where_clause) + "&$group=boro_nm"
)
url = f"{base_url}?{soql_query}"

r = requests.get(url)
data = r.json()

df_by_borough = pd.DataFrame(data)

print(df_by_borough.columns)

# Find the count column (Socrata might name it 'count' or 'count_') (This was helped by AI)
count_col = [col for col in df_by_borough.columns if "count" in col][0]

# Convert count to integer
df_by_borough[count_col] = df_by_borough[count_col].astype(int)

# Rename columns for clarity
df_by_borough = df_by_borough.rename(
    columns={"boro_nm": "Borough", count_col: "Total_Incidents"}
)

# Show result
print(df_by_borough)


Index(['boro_nm', 'count'], dtype='object')
         Borough  Total_Incidents
0          BRONX          1057446
1       BROOKLYN          1633175
2      MANHATTAN          1398299
3         (null)            36503
4         QUEENS           967593
5  STATEN ISLAND           226280


In [18]:
df_by_borough.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 2 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Borough          6 non-null      object
 1   Total_Incidents  6 non-null      int64 
dtypes: int64(1), object(1)
memory usage: 228.0+ bytes


# Step 2: Clean Data

I am now going to clean this data to select only the variables that I need to work with and make sure that it is usable for my later visualizations.

**2025 NYC Population Age Data**

In [19]:
# First, I will filter for borough to get rid of the rows with population totals and I will select for the year 2025 because it is the most recent data.

NYC_pop_age_columns_keep = ["Borough", "Age", "2025"]

NYC_pop_age_2025 = NYC_pop_age[NYC_pop_age_columns_keep]

boroughs = ["Staten Island", "Queens", "Manhattan", "Brooklyn", "Bronx"]

NYC_pop_age_2025 = NYC_pop_age_2025[NYC_pop_age_2025["Borough"].isin(boroughs)]

NYC_pop_age_2025.head()

,Borough,Age,2025
17,Bronx,0-4,"111,127"
18,Bronx,15-19,"103,114"
19,Bronx,20-24,"107,423"
20,Bronx,25-29,"117,887"
21,Bronx,30-34,"122,434"


In [ ]:
# Now I will combine the Age rows into three groups: child (0-19), adult (20-64), and senior (65+).

import numpy as np

# This should strip out an extra whitespaces.
NYC_pop_age_2025["Age"] = NYC_pop_age_2025["Age"].str.strip()

# I am converting the 2025 population column from an object to an integer and removing the commas so it will add nicely when I combine age groups.
NYC_pop_age_2025["2025"] = pd.to_numeric(
    NYC_pop_age_2025["2025"].astype(str).str.replace(",", "")
)

# I am making the three popualtion groups from the age ranges provided.
child = ["0-4", "5-9", "10-14", "15-19"]

adult = [
    "20-24",
    "25-29",
    "30-34",
    "35-39",
    "40-44",
    "45-49",
    "50-54",
    "55-59",
    "60-64",
]

senior = ["65-69", "70-74", "75-79", "80-84", "85+"]

# I am making a new variable with all three ranges to convert the rows later.
age_groups = child + adult + senior

# Here is where I am seleting the rows as the three age groups. After recieved an error, I added the default='unknown' to account for data that was not in the three groups.
NYC_pop_age_2025["Age_Group"] = np.select(
    [
        NYC_pop_age_2025["Age"].isin(child),
        NYC_pop_age_2025["Age"].isin(adult),
        NYC_pop_age_2025["Age"].isin(senior),
    ],
    ["child", "adult", "senior"],
    default="unknown",
)

# I am using group by to combine the total of the age group rows by borough.
NYC_pop_age_2025_grouped = NYC_pop_age_2025.groupby(
    ["Borough", "Age_Group"], as_index=False
)["2025"].sum()

# Change the rows displaying 'unknown' to say'Population Total".
NYC_pop_age_2025_grouped["Age_Group"] = NYC_pop_age_2025_grouped["Age_Group"].replace(
    "unknown", "population total"
)

# I am viewing the cleaned data.
NYC_pop_age_2025_grouped.head()

,Borough,Age_Group,2025
0,Bronx,adult,869330
1,Bronx,child,424165
2,Bronx,senior,191780
3,Bronx,population total,1485275
4,Brooklyn,adult,1610181


In [ ]:
# This is used to check what data type 2025 is because it would not add up correctly until I converted it to an integer.
NYC_pop_age_2025_grouped.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Borough    20 non-null     object
 1   Age_Group  20 non-null     object
 2   2025       20 non-null     int64 
dtypes: int64(1), object(2)
memory usage: 612.0+ bytes


In [ ]:
# Now I need to convert the population counts into proportions of the total population.

# Here I am seperating out the total popualtion count from the age ranges and pivoting them to a new column by borough.
pop_total = NYC_pop_age_2025_grouped[
    NYC_pop_age_2025_grouped["Age_Group"] == "population total"
][["Borough", "2025"]].rename(columns={"2025": "total_pop"})
ages = NYC_pop_age_2025_grouped[
    NYC_pop_age_2025_grouped["Age_Group"] != "population total"
]

# I am merging the totals back to age-group rows
merged = ages.merge(pop_total, on="Borough", how="left")

# I am making a column for the proportions by dividing populations by the total and multiplying by 100.
merged["proportion"] = (merged["2025"] / merged["total_pop"]) * 100

print(merged)

          Borough Age_Group     2025  total_pop  proportion
0           Bronx     adult   869330    1485275   58.529902
1           Bronx     child   424165    1485275   28.558011
2           Bronx    senior   191780    1485275   12.912087
3        Brooklyn     adult  1610181    2706246   59.498693
4        Brooklyn     child   711926    2706246   26.306773
5        Brooklyn    senior   384139    2706246   14.194534
6       Manhattan     adult  1101820    1661574   66.311822
7       Manhattan     child   292800    1661574   17.621845
8       Manhattan    senior   266954    1661574   16.066332
9          Queens     adult  1459417    2353431   62.012313
10         Queens     child   543205    2353431   23.081408
11         Queens    senior   350809    2353431   14.906279
12  Staten Island     adult   286460     493440   58.053664
13  Staten Island     child   119838     493440   24.286235
14  Staten Island    senior    87142     493440   17.660101


**2025 NYC 911 Call Data**

For the purpose of this assignment, I am going to display the code that I wrote myself to clean the inital 911 data that I loaded in. This code is shown below:

Here I want to select on the columns with the incident date and the borough.

df_911_subset = df_911[["incident_date", "boro_nm"]]

df_911_subset

Now I want to convert the incident column to only the year so I can select for data from 2025.

df_911_subset["incident_date"] = pd.to_datetime(
    df_911_subset["incident_date"], errors="coerce"
)

df_911_subset["Year"] = df_911_subset["incident_date"].dt.year

df_2025 = df_911_subset[df_911_subset["Year"] == 2025]

df_2025.head()

Now I am going to group by borough and filter out the nulls.

Count incidents per borough
df_911_grouped = df_2025.groupby("boro_nm").size().reset_index(name="incident_count")

df_911_grouped.head()

## Step 3: Data Visualization

**2025 NYC Borough Populations by Age**

In [32]:
# I am importing plotly to create my visualization.
import plotly.express as px

# Through the power of google, I was able to figure out how to change the bars in a grouped histogram by using colors as a variable and then adding it into my histogram code.
colors = ["#e377c2", "#B2BEE3", "#AED2AD", "#F0E597"]

fig = px.histogram(
    NYC_pop_age_2025_grouped,
    x="Borough",
    y="2025",
    color="Age_Group",
    barmode="group",
    height=600,
    title="2025 NYC Borough Populations by Age",
    color_discrete_sequence=colors,
)
# Beacuse this is technically a histogram, which shouldn't have a label for the y-axis, I had to add the titles under update_layout to skirt around the error message.
fig.update_layout(
    xaxis_title="Boroughs by Age Group", yaxis_title="2025 Population Total"
)
fig.show()

From this visualization, I can see that Brooklyn has the largest number of 65+ citizens. 

The breakdown is as follows:

In [ ]:
NYC_senior = NYC_pop_age_2025_grouped[NYC_pop_age_2025_grouped["Age_Group"] == "senior"]

NYC_senior = NYC_senior.sort_values(by="2025", ascending=False)

NYC_senior

,Borough,Age_Group,2025
6,Brooklyn,senior,384139
14,Queens,senior,350809
10,Manhattan,senior,266954
2,Bronx,senior,191780
18,Staten Island,senior,87142


In [36]:
# I am importing plotly to create my visualization.
import plotly.express as px

colors = ["#e377c2", "#B2BEE3", "#AED2AD", "#F0E597"]

fig = px.histogram(
    merged,
    x="Borough",
    y="proportion",
    color="Age_Group",
    barmode="group",
    height=600,
    title="2025 NYC Borough Population Proportions by Age Range",
    color_discrete_sequence=colors,
)

fig.update_layout(
    xaxis_title="Boroughs by Age Group", yaxis_title="2025 Total Population Proportion"
)
fig.show()

Here we can see which boroughs actually have the largest concentration of 65+ populations in relation to the total borough population. The data is as follows:

In [39]:
NYC_senior_pro = merged[merged["Age_Group"] == "senior"]

NYC_senior_pro = NYC_senior_pro.sort_values(by="proportion", ascending=False)

NYC_senior_pro

,Borough,Age_Group,2025,total_pop,proportion
14,Staten Island,senior,87142,493440,17.660101
8,Manhattan,senior,266954,1661574,16.066332
11,Queens,senior,350809,2353431,14.906279
5,Brooklyn,senior,384139,2706246,14.194534
2,Bronx,senior,191780,1485275,12.912087


From this table we can see that Staten Island and Manhattan have the highest 65+ populations in relation to their total populations.

**2025 NYC 911 Calls by Borough**

In [ ]:
fig = px.bar(
    df_by_borough,
    x="Borough",
    y="Total_Incidents",
    color_discrete_sequence=["#B2BEE3"],
    title="2025 NYC 911 Calls by Borough",
)

fig.update_layout(xaxis_title="NYC Borough", yaxis_title="911 Call Total")

fig.show()

## Step 3: Bringing It All Together

should i select for the three highest boroughs with 911 call volume then compare the age ranges?

should i do age ranges by borough?

NYC borough with highest proportion of seniors with highest proportion of 911 calls.



## Conlusion

counfounding variables?

higher populations? more traffic?

nyc 911 only q1-3


- merge both data frames
- do proportion for 911 calls
- graph
- make scatterplot
- conclusion
